In [3]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
# Set up Selenium options and driver
options = Options()
options.add_argument("--disable-gpu")
# options.add_argument("--headless")  # Optional: Run in headless mode

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Navigate to the page
base_url = "https://www.nykaa.com/brands/colorbar/c/642?"
params = "intcmp=brand_menu,most_viewed,colorbar"
paginationNumber = 1

# Allow initial time for the page to load
time.sleep(5)

# Scroll incrementally until all products are loaded
scroll_pause_time = 2  # Wait time after each scroll

# Lists to store data
productLinks = []
products = []

while True:

    if paginationNumber == 1:
        url = f"{base_url}{params}"
    else:
        url = f"{base_url}page_no={paginationNumber}&sort=popularity&intcmp=brand_menu,most_viewed,colorbar&eq=desktop"
    
    # Load the Page
    driver.get(url)
    print(f"Scraping Page {paginationNumber}: {url}")
    time.sleep(3)
    
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

    # Wait until the products are loaded
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.productWrapper.css-17nge1h'))
        )
    except TimeoutException:
        print(f"Scraped the Total number of pages: {paginationNumber - 1}")
        break

    # Parse the page content with BeautifulSoup
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')
    
    # Find all product items
    deygaOrganicProducts = soup.findAll('div', class_='productWrapper css-17nge1h')
    if not deygaOrganicProducts:
        print("No Products found on this page.")
        break

    # Extract product links
    for product in deygaOrganicProducts:
        # Extract the Stock Status
        stockTag = product.find('span', class_ = "css-lg5xc9")
        if stockTag:
            stock = stockTag.get_text(strip = True)
        else:
            stock = "In Stock"

        # Extract the Product Highlight
        highlightTag = product.find("li", class_ = "custom-tag css-1bse542")
        if highlightTag:
            highlight = highlightTag.get_text(strip = True)
        else:
            highlight = "None"

        # Extract product link
        linkTag = product.find('a', href=True)
        if linkTag:
            productLink = linkTag['href']
            # Check if the link is a relative URL and add the base URL if necessary
            if not productLink.startswith("http"):  # Relative URL
                productLink = "https://www.nykaa.com" + productLink
            productLinks.append({"link" : productLink, "stock" : stock, "highlight" : highlight})
                
    for productData in productLinks:
        
        link = productData["link"]
        stock = productData["stock"]
        highlight = productData["highlight"]
        
        driver.get(link)
        time.sleep(3)  # Allow time for the page to load

        # Parse individual product page
        productPageContent = driver.page_source
        productSoup = BeautifulSoup(productPageContent, 'html.parser')
    
        # Extract Product Name
        productTag = productSoup.find('h1', class_='css-1gc4x7i')
        if productTag:
            prodName = productTag.get_text(strip=True)
        else:
            prodName = "No Description Available"
    
        # Extract Ratings
        ratingTag = productSoup.find('div', class_="css-1m0y15j")
        if ratingTag:
            ratingText = ratingTag.find('div', class_="css-m6n3ou")
            if ratingText:
                ratings = ratingText.get_text(strip = True)
            else:
                ratings = "N/A"
        else:
            ratings = "N/A"

        # Extract the Ratings & Reviews Count
        ratingCountTag = productSoup.find('div', class_ = "css-1eip5u4")
        if ratingCountTag:
            ratingCount = ratingCountTag.get_text(strip = True)
        else:
            ratingCount = "None"

        # Extract the Offer Price and Original Price
        productDiv = productSoup.find('div', class_ = "css-1d0jf8e")

        # Extract the first two (span) tags
        if productDiv:
            spanTags = productDiv.find_all('span')
            # print(len(spanTags))

            originalPrice = "N/A"
            offerPrice = "N/A"

            if len(spanTags) == 2:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = "N/A"
            elif len(spanTags) == 4:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = spanTags[2].get_text(strip = True)

            # Print the Values
            # print(f"Original Price: {originalPrice}, Offer Price: {offerPrice}")

        else:
            print("Product div not found, defaulting to Unavailable for prices")

        # Extract the Discount
        discountTag = productSoup.find('span', class_ = "css-bhhehx")
        if discountTag:
            discount = discountTag.get_text(strip = True)
        else:
            discount = "No Discount"

        products.append({
            "Product Brand" : "Colorbar",
            "Product Name" : prodName,
            "Product Rating" : ratings,
            "Product Rating & Review Count" : ratingCount,
            "Product Original Price" : originalPrice, 
            "Product Offer Price" : offerPrice, 
            "Product Discount" : discount,
            "Product Highlight" : highlight,
            "Product Stock Status" : stock
        })

    # Clear the product links list to avoid the duplication
    productLinks.clear()

    paginationNumber += 1 # Increment the Page Number
    time.sleep(5)

# Close the browser after scraping
driver.quit()

Scraping Page 1: https://www.nykaa.com/brands/colorbar/c/642?intcmp=brand_menu,most_viewed,colorbar
Scraping Page 2: https://www.nykaa.com/brands/colorbar/c/642?page_no=2&sort=popularity&intcmp=brand_menu,most_viewed,colorbar&eq=desktop
Scraping Page 3: https://www.nykaa.com/brands/colorbar/c/642?page_no=3&sort=popularity&intcmp=brand_menu,most_viewed,colorbar&eq=desktop
Scraping Page 4: https://www.nykaa.com/brands/colorbar/c/642?page_no=4&sort=popularity&intcmp=brand_menu,most_viewed,colorbar&eq=desktop
Scraping Page 5: https://www.nykaa.com/brands/colorbar/c/642?page_no=5&sort=popularity&intcmp=brand_menu,most_viewed,colorbar&eq=desktop
Scraping Page 6: https://www.nykaa.com/brands/colorbar/c/642?page_no=6&sort=popularity&intcmp=brand_menu,most_viewed,colorbar&eq=desktop
Scraping Page 7: https://www.nykaa.com/brands/colorbar/c/642?page_no=7&sort=popularity&intcmp=brand_menu,most_viewed,colorbar&eq=desktop
Scraping Page 8: https://www.nykaa.com/brands/colorbar/c/642?page_no=8&sort=po

In [5]:
print(f"Products: \n {products}")

Products: 
 [{'Product Brand': 'Colorbar', 'Product Name': 'Colorbar Illuminating Base Combo(2 pcs)', 'Product Rating': '4.4/5', 'Product Rating & Review Count': '31121ratings&1781reviews', 'Product Original Price': '₹1645', 'Product Offer Price': '₹1234', 'Product Discount': '25% Off', 'Product Highlight': 'None', 'Product Stock Status': 'In Stock'}, {'Product Brand': 'Colorbar', 'Product Name': 'Colorbar Shine N Shimmer Complete Makeup Kit(9 Pcs)', 'Product Rating': '4.5/5', 'Product Rating & Review Count': '18ratings&2reviews', 'Product Original Price': '₹9234', 'Product Offer Price': '₹4802', 'Product Discount': '48% Off', 'Product Highlight': 'None', 'Product Stock Status': 'In Stock'}, {'Product Brand': 'Colorbar', 'Product Name': 'Colorbar Kissproof Lipstick - Wild Peach(3g)', 'Product Rating': '4.1/5', 'Product Rating & Review Count': '4496ratings&270reviews', 'Product Original Price': '₹1200', 'Product Offer Price': '₹600', 'Product Discount': '50% Off', 'Product Highlight': '

In [6]:
colorbar  = pd.DataFrame(products)
colorbar

,Product Brand,Product Name,Product Rating,Product Rating & Review Count,Product Original Price,Product Offer Price,Product Discount,Product Highlight,Product Stock Status
0,Colorbar,Colorbar Illuminating Base Combo(2 pcs),4.4/5,31121ratings&1781reviews,₹1645,₹1234,25% Off,None,In Stock
1,Colorbar,Colorbar Shine N Shimmer Complete Makeup Kit(9...,4.5/5,18ratings&2reviews,₹9234,₹4802,48% Off,None,In Stock
2,Colorbar,Colorbar Kissproof Lipstick - Wild Peach(3g),4.1/5,4496ratings&270reviews,₹1200,₹600,50% Off,None,In Stock
3,Colorbar,Colorbar Kiss Proof Lip Stain - Haute Latte - ...,4.2/5,6146ratings&1047reviews,₹999,₹769,23% Off,BESTSELLER,In Stock
4,Colorbar,Colorbar Perfect Match Primer(30ml),4.4/5,16015ratings&1720reviews,₹950,₹741,22% Off,BESTSELLER,In Stock
...,...,...,...,...,...,...,...,...,...
363,Colorbar,Colorbar Co-Earth Vitamin C Under Eye Cream(20ml),4.5/5,24ratings&2reviews,₹449,₹225,50% Off,None,Out Of Stock
364,Colorbar,Colorbar Co-Earth Ubtan Night Face Mask(100 g),1/5,4ratings,₹799,₹400,50% Off,None,Out Of Stock
365,Colorbar,Colorbar Co-Earth Tea Tree Face Mask(100 g),4.3/5,9ratings,₹575,₹288,50% Off,None,Out Of Stock
366,Colorbar,Colorbar Co-Earth Tea Tree Foaming Face Wash(1...,4.3/5,33ratings,₹499,₹250,50% Off,None,Out Of Stock


In [7]:
# colorbar.to_csv("C:/Users/Logiya Vidhyapathy/Documents/KGISL Data Science/Project Files and Documents/Capstone Project/Colorbar Products_Nykaa.csv", index = False)